<a href="https://colab.research.google.com/github/gabselbach/TCC-implementacoes/blob/master/AcentuaF%C3%A1cil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [ ]:
!pip3 install Dicio
import os
import re
import pandas as pd
import time
import requests
import spacy
import nltk
from spacy.tokenizer import Tokenizer
from bs4 import BeautifulSoup
from unicodedata import normalize
from re import match as re_match
from re import compile as re_compile
import spacy
import spacy.cli
import ast
from dicio import Dicio
dicio = Dicio()
nltk.download('stopwords')
spacy.cli.download('pt_core_news_sm')
nlp = spacy.load("pt_core_news_sm")
from openpyxl.workbook import Workbook
nltk.download('stopwords')
!pip install python-Levenshtein
from Levenshtein import distance

# busca no DICIO

In [ ]:
def isdigit(s):
    comp = re_compile("^\d+?\.\d+?$")
    if comp.match(s) is None:
        return s.isdigit()
    return True


def faz_busca2(token):
    acento = re.compile('à|[á-ú]|ê|ô|ã|õ|í')
    dataNova = []
    for k in token:
        t = 0
        busca = k.lower()
        if(len(busca) > 2 and (not isdigit(busca))):
            busca = re.sub("[?|;|,*|.*]", "", busca)
            page = requests.get('https://www.dicio.com.br/'+busca+'/')
            soup = BeautifulSoup(page.text, 'html.parser')
            pDicio = soup.find('h1').text
            if(not acento.search(pDicio) or pDicio==busca or pDicio.lower()=="não encontrada"):
              temp = {
                'PALAVRAANT': k.lower(),
                'PALAVRADICIO': pDicio.lower(),
                'SILABA':'',
                'CLASSE': '',
                'FORTE': '',
                'MONOSSILABA': 0,
                'ACENTO': 0,
                'CORREC':0,
                'TIPOCORREC':0,
                'EXCEÇAO':0,
                'REGRAVERB':0,
                'REGRANaoVERB': 0
              }
              dataNova.append(temp)
            else: 
                texto = soup.find_all('p', {'class': 'adicional'})
                try:
                    n = re.search(r"silábica: .+</b>", str(texto))
                    if(n == None):
                        temp = {
                            'PALAVRAANT': k.lower(),
                            'PALAVRADICIO': pDicio.lower(),
                            'SILABA': 'FALTA',
                            'CLASSE': '',
                            'FORTE': '',
                            'MONOSSILABA': 0,
                            'ACENTO': 1,
                            'CORREC': 0,
                            'TIPOCORREC': 'DICIO+SEPARADOR',
                            'EXCEÇAO': 0,
                            'REGRAVERB': 0,
                            'REGRANaoVERB': 0
                        }
                        dataNova.append(temp)
                    else:
                      novo = str(re.sub('<[^>]+?>', '', n.group(0))).split(':')
                      silaba = novo[1]
                      silsepara = silaba.split('-')
                      if(len(silsepara) == 1):
                        mono = 1
                      else:
                        mono = 0
                      oxi = ''
                      paro = ''
                      propa = ''
                      classe = ''
                      forte = ''
                      aux = 0
                      for j in range(len(silsepara)-1, -1, -1):
                        aux += 1
                        if(acento.search(silsepara[j]) and aux == 1):
                            classe = 'oxítona'
                            forte = silsepara[j]
                        elif(acento.search(silsepara[j]) and aux == 2):
                            classe = 'paroxítona'
                            forte = silsepara[j]
                        elif(acento.search(silsepara[j]) and aux == 3):
                            classe = 'proparoxítona'
                            forte = silsepara[j]
                      temp = {
                        'PALAVRAANT': k.lower(),
                        'PALAVRADICIO': pDicio.lower(),
                        'SILABA': silaba,
                        'CLASSE': classe,
                        'FORTE': forte,
                        'MONOSSILABA': mono,
                        'ACENTO': 1,
                        'CORREC': 1,
                        'TIPOCORREC': 'dicionário',
                        'EXCEÇAO': 0,
                        'REGRAVERB': 0,
                        'REGRANaoVERB': 0
                      }
                      dataNova.append(temp)
                except:
                  print("")
        else:
            temp = {
                'PALAVRAANT': k.lower(),
                'PALAVRADICIO': 'NAN',
                'SILABA': '',
                'CLASSE': '',
                'FORTE': '',
                'MONOSSILABA': '',
                'ACENTO': 0,
                'CORREC': 0,
                'TIPOCORREC': 'NAN',
                'EXCEÇAO': 0,
                'REGRAVERB': 0,
                'REGRANaoVERB': 0
            }
            dataNova.append(temp)
    return dataNova

# busca no VOP

In [ ]:
def faz_busca(token):
    dataNova = []
    valorTime = 0
    count = 0
    acento = re.compile('à|[á-ú]|ê|ô|ã|õ|í')
    for k in token:
        t = 0
        busca = normalize('NFKD', str(k).lower()).encode(
            'ASCII', 'ignore').decode('ASCII')
        if(len(busca) > 2 and (not isdigit(busca))):
            page = requests.get(
                'http://www.portaldalinguaportuguesa.org/index.php?action=syllables&act=list&search='+busca)
            soup = BeautifulSoup(page.text, 'html.parser')
            palavras = soup.find_all('td', {'title': 'Palavra'})
            classe = ''
            if(not palavras or len(k) == 1):
                temp = {
                    'PALAVRAANT': k.lower(),
                    'PALAVRAVOP': 'NAN',
                    'SILABA': 'NAN',
                    'CLASSE': 'NAN',
                    'FORTE': 'NAN',
                    'MONOSSILABA': 0,
                    'ACENTO': 1,
                    'CORREC':0,
                    'TIPOCORREC':''
                }
                dataNova.append(temp)
            else:
                aux = 0
                for i in palavras:
                    link = re.sub('<[^>]+?>', '', str(i.find('a')))
                    pala = i.text.replace(" ", '').split('\n')[0]
                    palavraNormalizada = normalize('NFKD', link).encode(
                        'ASCII', 'ignore').decode('ASCII').lower()
                    if(palavraNormalizada == busca):
                        if(not acento.search(str(link)) or k.lower()==str(link).lower()):
                            t = 1
                            temp = {
                                'PALAVRAANT': k.lower(),
                                'PALAVRAVOP': 'NAN',
                                'SILABA': 'NAN',
                                'CLASSE': 'NAN',
                                'FORTE': 'NAN',
                                'MONOSSILABA': 0,
                                'ACENTO': 0,
                                'CORREC':0,
                                'TIPOCORREC':''
                            }
                            dataNova.append(temp)
                            break
                        forte = re.sub('<[^>]+?>', '', str(i.find('u')))
                        silaba = pala.split(')')[1]
                        silsepara = silaba.split('·')
                        if(len(silsepara) == 1):
                            mono = 1
                        else:
                            mono = 0
                        oxi = ''
                        paro = ''
                        propa = ''
                        for j in range(len(silsepara)-1, -1, -1):
                            if(aux == 0):
                                oxi = silsepara[j] + oxi
                            elif(aux == 1):
                                paro = silsepara[j] + paro
                            elif(aux == 2):
                                propa = silsepara[j] + propa
                            aux += 1
                        if(oxi == forte):
                            classe = 'oxítona'
                        elif(paro == forte):
                            classe = 'paroxítona'
                        elif(propa == forte):
                            classe = 'proparoxítona'
                        temp = {
                            'PALAVRAANT': k.lower(),
                            'PALAVRAVOP': link.lower(),
                            'SILABA': silaba,
                            'CLASSE': classe,
                            'FORTE': normalize('NFKD', str(forte)).encode('ASCII', 'ignore').decode('ASCII'),
                            'MONOSSILABA': mono,
                            'ACENTO': 1,
                            'CORREC':1,
                            'TIPOCORREC':'dicionário'
                        }
                        dataNova.append(temp)
                        t = 1
                        break
                    if(t == 1):
                        break
                if(t != 1):
                    temp = {
                        'PALAVRAANT': k.lower(),
                        'PALAVRAVOP': 'NAN',
                        'SILABA': 'NAN',
                        'CLASSE': 'NAN',
                        'FORTE': 'NAN',
                        'MONOSSILABA': 0,
                        'ACENTO': 1,
                        'CORREC':0,
                        'TIPOCORREC':''
                    }
                    dataNova.append(temp)
        else:
            if(isdigit(busca)):
                x = 1
            else:
                temp = {
                    'PALAVRAANT': k.lower(),
                    'PALAVRAVOP': 'NAN',
                    'SILABA': 'NAN',
                    'CLASSE': 'NAN',
                    'FORTE': 'NAN',
                    'MONOSSILABA': 0,
                    'ACENTO': 0,
                    'CORREC':0,
                    'TIPOCORREC':''
                }

                dataNova.append(temp)
    return dataNova

# crawler para o seaparador e função para pegar a sílaba forte

In [ ]:
def separador(dicionario):
  for (l,row) in dicionario.iterrows(): 
    if(row['SILABA']=='FALTA' and (row['PALAVRADICIO']!='NAN' and row['PALAVRADICIO']!='não encontrada') ):
      page = requests.get("https://www.separaremsilabas.com/index.php?lang=index.php&p="+row['PALAVRADICIO']+"&button="+"Separa%C3%A7%C3%A3o+das+s%C3%ADlabas")
      soup = BeautifulSoup(page.text, 'html.parser')
      texto = str(soup.find('font', {'color': '#0018BF'}))
      texto = re.sub('<[^>]+?>', '',texto)
      dicionario.at[l,'SILABA']=texto
      dicionario.at[l,'ACENTO']=1
      dicionario.at[l,'CORREC']=1
      dicionario.at[l,'TIPOCORREC']='dicionario+separador'
      temp = pegaForte(texto)
      dicionario.at[l,'CLASSE']=temp['classe']
      dicionario.at[l,'FORTE']=temp['forte']
  return dicionario

In [ ]:
def pegaForte(silaba):
  acento = re.compile('à|[á-ú]|ê|ô|ã|õ|í')
  silsepara = silaba.split('-')
  oxi = ''
  paro = ''
  propa = ''
  forte=''
  classe=''
  aux=0
  for j in range(len(silsepara)-1, -1, -1):
    aux += 1
    if(acento.search(silsepara[j]) and aux==1):
      classe = 'oxítona'
      forte = silsepara[j]
    elif(acento.search(silsepara[j]) and aux==2):
      classe = 'paroxítona'
      forte = silsepara[j]
    elif(acento.search(silsepara[j]) and aux==3):
      classe = 'proparoxítona'   
      forte = silsepara[j]
  t = {
      'forte':forte,
       'classe':classe
  }
  return t